In [1]:
import pandas as pd   
import pandas_gbq as pgbq 
import os
import re
import sys

In [2]:
path_note = os.getcwd()
path_src = re.sub(r'(.)notebooks','',path_note)
sys.path.append(os.path.abspath(f'{path_src}/'))

In [3]:
from fin_api import consulta_gcp
strsql = '''   select descricao
        , dt_recebido 
        , dt_mes_base
        , valor_recebido
        , row_number() over (partition by descricao,	dt_recebido	, cast(valor_recebido as string),	dt_mes_base order by process_time desc) ordem
        , row_number() over (partition by 	dt_mes_base	 order by dt_recebido desc) ordem_recebimento
    from `devsamelo2.dev_domestico.recebido_2024_excel`
    '''

df = consulta_gcp(strsql)

In [10]:
df[df['valor_recebido']>=2]

,descricao,dt_recebido,dt_mes_base,valor_recebido,ordem,ordem_recebimento
3,PIX TRANSF Matheus25/01,2024-01-26,2024-01-01,50.00,1,4
11,PAGTO SALARIO,2024-01-15,2024-01-01,4115.42,1,12
14,PIX TRANSF MICHELL10/01,2024-01-10,2024-01-01,310.00,1,15
18,PAGTO SALARIO,2024-01-05,2024-01-01,2409.98,1,19
22,PIX TRANSF MICHELL20/02,2024-02-20,2024-02-01,500.00,1,1
25,PAGTO EM CONTA CORRENTE,2024-02-15,2024-02-01,4115.42,1,4
29,PAGTO SALARIO,2024-02-05,2024-02-01,2889.18,1,8
31,TED 104.0000CAIXA ECON F,2024-02-01,2024-02-01,152.76,1,10


In [9]:
df[df['valor_recebido']>=1]['valor_recebido'].sum()
#df[['dt_mes_base', 'valor_recebido', 'valor_custo_real', 'saldo_real']].sort_values('dt_mes_base', ascending=False) #.reset_index(inplace=True , drop=True)#

14545.82

In [14]:
df[df['descricao'].str.contains(r'PAGTO[^\\b]+\w+')==True]['valor_recebido'].sum()
#.str.contains(r'SALDO[^\\b]+\w|APL[^\\b]APLIC[^\\b]AUT[^\\b]MAIS|RES[^\\b]APLIC[^\\b]AUT[^\\b]MAIS|REND[^\\b]PAGO[^\\b]APLIC[^\\b]AUT[^\\b]MAIS')==False]

13530.0

In [15]:
df[df['descricao'].str.contains(r'PAGTO[^\\b]+\w+')==False]['valor_recebido'].sum()


1020.0